In [5]:
//////////////////////// eml run n bloc 3  ///////////////////// 
import cv2
import tensorflow as tf

# Load the TensorFlow model
model = tf.keras.models.load_model('C:\\Users\\USER\\Desktop\\MASTER-QL\\Python\\haarcascade_fullbody.xml')

# Open the video file
video = cv2.VideoCapture('C:\\Users\\USER\\Desktop\\MASTER-QL\\Python\\Walking_video.mp4')

# Initialize counters for the number of people detected in each frame
people_count = 0
frame_count = 0

# Loop through each frame of the video
while True:
  # Read the next frame
  success, frame = video.read()
  if not success:
    break

  # Preprocess the frame for the model
  frame = cv2.resize(frame, (300, 300))
  frame = frame.astype('float32') / 255.0
  frame = tf.expand_dims(frame, axis=0)

  # Run the model and get the predictions
  predictions = model.predict(frame)

  # Extract the bounding boxes and classes for the people detected
  boxes, classes, scores = tf.image.combined_non_max_suppression(
      boxes=predictions[:, :, :4],
      scores=predictions[:, :, 4],
      max_output_size_per_class=1,
      max_total_size=1,
      iou_threshold=0.5,
      score_threshold=0.5
  )

  # Increment the counters based on the number of people detected
  people_count += len(boxes)
  frame_count += 1

# Print the total number of people detected over the entire video
print(f'Number of people detected: {people_count}')


ImportError: Filepath looks like a hdf5 file but h5py is not available. filepath=C:\Users\USER\Desktop\MASTER-QL\Python\haarcascade_fullbody.xml

In [11]:
import cv2
from sklearn.cluster import KMeans

# Open the video file
video = cv2.VideoCapture('C:\\Users\\USER\\Desktop\\MASTER-QL\\Python\\Walking_video.mp4')

# Read the first frame and create a background model
success, background = video.read()
if not success:
  raise Exception('Failed to read the video file')

# Convert the background to grayscale and blur it
background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
background = cv2.GaussianBlur(background, (21, 21), 0)

# Initialize counters for the number of people detected in each frame
people_count = 0
frame_count = 0

# Loop through each frame of the video
while True:
  # Read the next frame
  success, frame = video.read()
  if not success:
    break

  # Convert the frame to grayscale and blur it
  frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  frame = cv2.GaussianBlur(frame, (21, 21), 0)

  # Subtract the background from the current frame
  foreground = cv2.absdiff(background, frame)

  # Threshold the foreground image to create a binary mask
  _, mask = cv2.threshold(foreground, 25, 255, cv2.THRESH_BINARY)

  # Dilate the mask to make it easier to identify connected components
  kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
  mask = cv2.dilate(mask, kernel, iterations=2)

  # Use connected component analysis to identify and count the blobs in the mask
  _, labels, stats, _ = cv2.connectedComponentsWithStats(mask)
  people = 0
  for label in range(1, labels):
    # Extract the bounding box for the blob
    x, y, w, h, area = stats[label]
    if area > 500:  # Filter out small blobs
      # Use K-Means clustering to separate the blob into two clusters
      image = frame[y:y+h, x:x+w]
      kmeans = KMeans(n_clusters=2)
      kmeans.fit(image.reshape((-1, 1)))
      # If the two clusters have significantly different intensities, assume the blob represents a person
      if abs(kmeans.cluster_centers_[0] - kmeans.cluster_centers_[1]) > 25:
        people += 1

  # Increment the counters based on the number of people detected
  people_count += people
  frame_count += 1

# Print the total number of people detected over the entire video
print(f'Number of people detected: {people_count}')


SyntaxError: can't assign to function call (1219027037.py, line 42)

In [ ]:
# Author: Addison Sears-Collins
# https://automaticaddison.com
# Description: Detect pedestrians in a video using the 
#   Histogram of Oriented Gradients (HOG) method
 
import cv2 # Import the OpenCV library to enable computer vision
import numpy as np # Import the NumPy scientific computing library
from imutils.object_detection import non_max_suppression # Handle overlapping
 
# Make sure the video file is in the same directory as your code
filename = 'C:\\Users\\USER\\Desktop\\MASTER-QL\\Python\\Video1.mp4'
file_size = (1920,1080) # Assumes 1920x1080 mp4
scale_ratio = 1 # Option to scale to fraction of original size. 
 
# We want to save the output to a video file
output_filename = 'detecting_on_street.mp4'
output_frames_per_second = 20.0
 
def main():
 
  # Create a HOGDescriptor object
  hog = cv2.HOGDescriptor()
     
  # Initialize the People Detector
  hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
     
  # Load a video
  cap = cv2.VideoCapture(filename)
 
  # Create a VideoWriter object so we can save the video output
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  result = cv2.VideoWriter(output_filename,  
                           fourcc, 
                           output_frames_per_second, 
                           file_size) 
     
  # Process the video
  while cap.isOpened():
         
    # Capture one frame at a time
    success, frame = cap.read() 
         
    # Do we have a video frame? If true, proceed.
    if success:
         
        # Resize the frame
      width = int(frame.shape[1] * scale_ratio)
      height = int(frame.shape[0] * scale_ratio)
      frame = cv2.resize(frame, (width, height))
             
      # Store the original frame
      orig_frame = frame.copy()
             
      # Detect people
      # image: a single frame from the video
      # winStride: step size in x and y direction of the sliding window
      # padding: no. of pixels in x and y direction for padding of 
      # sliding window
      # scale: Detection window size increase coefficient   
      # bounding_boxes: Location of detected people
      # weights: Weight scores of detected people
      # Tweak these parameters for better results
      (bounding_boxes, weights) = hog.detectMultiScale(frame, 
                                                       winStride=(16, 16),
                                                       padding=(4, 4), 
                                                       scale=1.05)
 
      # Draw bounding boxes on the frame
      for (x, y, w, h) in bounding_boxes: 
            cv2.rectangle(orig_frame, 
            (x, y),  
            (x + w, y + h),  
            (0, 0, 255), 
             2)
                         
      # Get rid of overlapping bounding boxes
      # You can tweak the overlapThresh value for better results
      bounding_boxes = np.array([[x, y, x + w, y + h] for (
                                x, y, w, h) in bounding_boxes])
             
      selection = non_max_suppression(bounding_boxes, 
                                      probs=None, 
                                      overlapThresh=0.45)
         
      # draw the final bounding boxes
      for (x1, y1, x2, y2) in selection:
        cv2.rectangle(frame, 
                     (x1, y1), 
                     (x2, y2), 
                     (0, 255, 0), 
                      4)
         
      # Write the frame to the output video file
      result.write(frame)
             
      # Display the frame 
      cv2.imshow("Frame", frame)    
 
      # Display frame for X milliseconds and check if q key is pressed
      # q == quit
      if cv2.waitKey(25) & 0xFF == ord('q'):
        break
         
    # No more video frames left
    else:
      break
             
  # Stop when the video is finished
  cap.release()
     
  # Release the video recording
  result.release()
     
  # Close all windows
  cv2.destroyAllWindows() 
 
main()